# Train Spectrogram Classifier

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import albumentations as A
from datetime import date
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.notebook import tqdm

from scripts.viz_tools import stretch_histogram, normalize

from scripts import dl_utils
from scripts.dl_utils import predict_spectrogram, rect_from_point
from scripts.nn_predict import make_predictions, visualize_predictions

np.random.seed(1)

## Create a Training Dataset
Outputs will be: `x_train`, `y_train`, `x_test`, `y_test`, and optionally, `x_holdout`, `y_holdout`. Holdout data is only positive.

In [ ]:
train_data_dir = '../data/training_data/spectrogram_patches_3mo-mosaics_2x-int/'
MOSAIC_PERIOD = 3
SPECTROGRAM_INTERVAL = 2
SPECTROGRAM_STEPS = 2 # WIP: Currently this is fixed - only pairs of mosaics are supported.
resolution = 32

data_files = [f for f in os.listdir(train_data_dir) if f.endswith('.pkl') and 'labels' not in f and 'polygons' not in f]
label_files = [f.split('s.pkl')[0] + '_labels.pkl' for f in data_files]
    

In [ ]:
patches = []
labels = []
for data, label in zip(data_files, label_files):
    with open(os.path.join(train_data_dir, data), 'rb') as f:
        file_data = pickle.load(f)
        patches += [[dl_utils.pad_patch(p[0], resolution), 
                     dl_utils.pad_patch(p[1], resolution)] for p in file_data if len(p) == 2]
        print(data)
    with open(os.path.join(train_data_dir, label), 'rb') as f:
        label_data = pickle.load(f)
        print(f"class {label_data[-1]}. {len(label_data)} samples")
        labels += label_data
patches = np.array(patches)
labels = np.array(labels)

print("Loaded", sum(labels == 1), "positive patches and", sum(labels == 0), "negative patches")

### Combine data and create train test split
Also expand dimensions to account for batches

In [ ]:
x = np.copy(patches)
y = np.copy(labels)

x, y = shuffle(x, y, random_state=42)
x = np.array([[dl_utils.unit_norm(p[0]), dl_utils.unit_norm(p[1])] for p in x])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

print("Num Train Samples:\t\t", len(x_train))
print("Num Test Samples:\t\t", len(x_test))
print(f"Percent Negative Train:\t {sum(y_train == 0.0) / len(y_train):.1%}")
print(f"Percent Negative Test:\t {sum(y_test == 0.0) / len(y_test):.1%}")
print(f"Input data shape: {x_train.shape}")

# Note: I am accustomed to assigning two classes for binary classification. 
# This habit comes from an issue in theano a long time ago, but I'm too superstitious to change it.
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
def augment_pairs(x, y, iterations=10, resolution=32):
    # create albumentations augmentation set
    # Play with params here: https://albumentations-demo.herokuapp.com/
    interpolation_mode = 1
    border_mode = 2
    aug = A.Compose([
        #A.RandomSizedCrop(min_max_height=(28, 30), height=resolution, width=resolution, p=1),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.15, 
                           scale_limit=0.15, 
                           rotate_limit=180, 
                           interpolation=interpolation_mode, 
                           border_mode=4, 
                           p=1),
        A.OneOf([
            #A.ElasticTransform(alpha=1, sigma=8, alpha_affine=20, interpolation=interpolation_mode, border_mode=border_mode), 
            A.GridDistortion(distort_limit=0.3, interpolation=interpolation_mode, border_mode=border_mode),
            A.OpticalDistortion(shift_limit=0.3, interpolation=interpolation_mode, border_mode=border_mode)
            ], p=0.8),
    ],
    additional_targets={'image_0': 'image', 'image_1': 'image'}
    )

    # It would be nice to have this as a generator passed to the train function
    # This was challenging though. Instead, I'm pregenerating a large dataset
    # and then training on it
    aug_x = []
    aug_y = []
    for i in range(iterations):
        print("iteration", i+1, "of", iterations)
        for pair, label in zip(x, y):
            pair_1, pair_2 = pair[0], pair[1]
            augmented = aug(image=pair_1, image_0=pair_1, image_1=pair_2)
            aug_x.append([augmented['image_0'], augmented['image_1']])
            aug_y.append(label)
        print("Augmented dataset now contains", len(aug_x), "pairs")
    aug_x = np.array(aug_x)
    aug_y = np.array(aug_y)
    
    return aug_x, aug_y

In [ ]:
x_aug, y_aug = augment_pairs(x_train, y_train, iterations=4, resolution=resolution)

In [ ]:
# Sanity check to confirm that pairs are augmented together
index = np.random.randint(0, len(x_train))
plt.figure(figsize=(3,3), dpi=150)
plt.subplot(2,2,1)
plt.title('Original 1')
plt.imshow(np.clip((x_train[index,0,:,:,3:0:-1] + 1) / 4, 0, 1))
plt.axis('off')
plt.subplot(2,2,2)
plt.title('Original 2')
plt.imshow(np.clip((x_train[index,1,:,:,3:0:-1] + 1) / 4, 0, 1))
plt.axis('off')
plt.subplot(2,2,3)
plt.title('Augmented 1')
plt.imshow(np.clip((x_aug[index,0,:,:,3:0:-1] + 1) / 4, 0, 1))
plt.axis('off')
plt.subplot(2,2,4)
plt.title('Augmented 2')
plt.imshow(np.clip((x_aug[index,1,:,:,3:0:-1] + 1) / 4, 0, 1))
plt.axis('off')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,12), facecolor=(1,1,1), dpi=150)
rand_indices = np.random.randint(0, len(x_train), size=64)
for index, (image, label) in enumerate(zip(x_aug[rand_indices], y_aug[rand_indices])):
    rgb = (image[0,:,:,3:0:-1] + 1) / 4
    plt.subplot(8, 8, index+1)
    plt.imshow(np.clip(rgb, 0, 1))
    if label[1] == 1:
        plt.title('Waste')
    else:
        plt.title('No Waste')
    plt.axis('off')
plt.suptitle('Data Augmentation Examples')
plt.tight_layout()
plt.show()

In [ ]:
x_train = np.concatenate((x_aug[:, 0, :, :, :], x_aug[:, 1, :, :, :]), 3)
x_test = np.concatenate((x_test[:, 0, :, :, :], x_test[:, 1, :, :, :]), 3)
print(x_train.shape)

## Create and Train a Model

In [ ]:
input_shape = x_train.shape[1:]
print("Input Shape:", input_shape)
input_img = keras.Input(input_shape)
x = layers.Conv2D(32, (5, 5), padding='same')(input_img)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(32, (5, 5), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(32, (5, 5), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, (4, 4), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(32, (4, 4), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(32, (4, 4), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, (3, 3), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(32, (3, 3), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(32, (3, 3), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation=layers.ELU())(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation=layers.ELU())(x)
x = layers.Dropout(0.5)(x)
out = layers.Dense(2, activation="softmax")(x)

model = keras.Model(input_img, out, name="classifier")

model.compile(loss="binary_crossentropy", #binary_crossentropy is default for two classes
                optimizer=keras.optimizers.Adam(epsilon=0.001), #haven't tried other optimizers, might be a good idea, but Adam is quite good usually
                metrics=["accuracy"])

version_number = '0.0.4'

current_date = date.today()
model_name = f"spectrogram_v{version_number}_{current_date.isoformat()}"

mcp_save = keras.callbacks.ModelCheckpoint(f"../models/{model_name}.hdf5", 
                                        save_best_only=True, monitor='accuracy', verbose=1, mode='max')

reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

train_accuracy = []
test_accuracy = []

model.summary()

In [ ]:
#train the model
EPOCHS = 32
BATCH_SIZE = 64

model.fit(x_train, 
    y_aug,
    validation_data=(x_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    #callbacks=[mcp_save],
    verbose = 1,
    shuffle=True
    )
#model.save("patch_ensemble/model_" + str(i) + ".hdf5")

In [ ]:
train_accuracy += model.history.history['accuracy']
test_accuracy += model.history.history['val_accuracy']
plt.figure(figsize=(8,5), dpi=100, facecolor=(1,1,1))
plt.plot(train_accuracy, label='Train Acc')
plt.plot(test_accuracy, c='r', label='Val Acc')
percent_negative = (sum(y_train == 0.0) / len(y_train))[1]
plt.plot([0, EPOCHS-1], [percent_negative, percent_negative], '--', c='gray', label='Baseline')
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Network Train and Val Accuracy')
plt.show()

In [ ]:
threshold = 0.8
print("Test Set Metrics:")
print(classification_report(y_test[:,1], model.predict(x_test)[:,1] > threshold, 
                            target_names=['No Waste', 'Waste']))



In [ ]:
threshold = 0.3
preds = model.predict(x_test)
preds_binary = preds[:,1] > threshold
# find missed examples
missed = np.where(preds_binary != y_test[:,1])[0]
print(f"{len(missed)} missed examples out of {len(y_test)} total test samples ({len(missed)/ len(y_test):.2%}) at a threshold of {threshold}")

In [ ]:
plt.figure(figsize=(12,12), facecolor=(1,1,1), dpi=150)
rand_indices = np.random.randint(0, len(missed), size=64)
for index, (image, pred, label) in enumerate(zip(x_test[missed[rand_indices]], preds[missed[rand_indices]], y_test[missed[rand_indices]])):
    rgb = (image[:,:,3:0:-1] + 1) / 4
    plt.subplot(8, 8, index+1)
    plt.imshow(np.clip(rgb, 0, 1))
    plt.title(f"{pred[1]:.3f}, {int(label[1])}")
    plt.axis('off')
plt.suptitle('Misclassified Samples')
plt.tight_layout()
plt.show()

## Save Model

In [ ]:
version_number = 'supervised-v2.0.0'

current_date = date.today()
model_name = f"patch_spectrogram_{version_number}_{current_date.isoformat()}"
assert not os.path.exists('../models/' + model_name + '.h5'), f"Model of name {model_name} already exists"

with open('../models/' + model_name + '_config.txt', 'w') as f:
    f.write(f'Image mosaic period: {MOSAIC_PERIOD}\n')
    f.write(f'Spectrogram interval, steps: {SPECTROGRAM_INTERVAL}, {SPECTROGRAM_STEPS}\n')
    f.write('Input Data:\n')
    [f.write(file + '\n') for file in data_files]
    f.write(f"\nBatch Size: {BATCH_SIZE}")
    f.write(f"\nTraining Epochs: {len(train_accuracy)}")
    f.write('\n\nClassification Report\n')
    f.write(classification_report(y_test[:,1], model.predict(x_test)[:,1] > threshold, 
                            target_names=['No Waste', 'Waste']))
model.save('../models/' + model_name + '.h5')